In [ ]:
!pip install pylatexenc --quiet
!pip install qiskit --quiet
!pip install graphviz --quiet

In [ ]:
!pip3 install torch
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

In [ ]:
import numpy as np
import pandas as pd
from qiskit import transpile
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import XGate
from qiskit.transpiler import PassManager, InstructionDurations
from qiskit.transpiler.passes import ALAPSchedule, ALAPScheduleAnalysis, DynamicalDecoupling, PadDynamicalDecoupling
from qiskit.visualization import timeline_drawer
from qiskit.dagcircuit import DAGCircuit
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.circuit.random import random_circuit
from qiskit.transpiler.passes.scheduling.padding.base_padding import BasePadding
from qiskit.providers.fake_provider import FakeLagosV2,FakeBoeblingenV2
from tqdm import tqdm
import networkx as nx
from torch_geometric.utils.convert import to_networkx, from_networkx
import torch

In [ ]:
matrix_sched = pd.DataFrame(columns=['input','output'])
for i in tqdm(range(0,500)):
  circ = random_circuit(10, 30, max_operands=2, measure=True)
  circ = circ.decompose().decompose()
  #circ.draw(output='mpl')
  cdag = circuit_to_dag(circ)
  cdag = cdag.to_networkx()
  l = nx.to_dict_of_lists(cdag)
  cdag = nx.Graph(l)
  cdag = from_networkx(cdag)

  backend = FakeBoeblingenV2()
  durations = backend.target.durations()

  circ_sched = transpile(circ, backend, scheduling_method="alap")
  pm = PassManager(ALAPScheduleAnalysis(durations))
  circ_sched = pm.run(circ_sched)
  dag_sched = circuit_to_dag(circ_sched)
  dag_sched = dag_sched.to_networkx()
  l = nx.to_dict_of_lists(dag_sched)
  dag_sched = nx.Graph(l)
  dag_sched = from_networkx(dag_sched)

  matrix_sched.loc[len(matrix_sched.index)] = [cdag, dag_sched] 
  #circ_sched.draw(output='mpl')


100%|██████████| 500/500 [09:51<00:00,  1.18s/it]


In [ ]:
matrix_sched.head()
from google.colab import files
matrix_sched.to_csv('dataset_scheduling_py.csv', encoding = 'utf-8-sig') 
files.download('dataset_scheduling_py.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['dataset_scheduling_py.csv']))

In [ ]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# Define the input and output arrays of graphs
input_graphs = matrix_sched['input'].tolist()
output_graphs = matrix_sched['output'].tolist()

# Create a list of tuples containing the input and output graphs
data_list = [(input_graph, output_graph) for input_graph, output_graph in zip(input_graphs, output_graphs)]

# Convert the list of tuples into a list of PyTorch Geometric Data objects
data_list = [Data(x=input_graph, y=output_graph) for input_graph, output_graph in data_list]

# Create a PyTorch Geometric data loader
dataloader = DataLoader(data_list, batch_size=32, shuffle=True)

In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

#class GCN(torch.nn.Module):
#    def __init__(self):
#        super().__init__()
#        self.conv1 = GCNConv(dataset.num_node_features, 16)
#        self.conv2 = GCNConv(16, dataset.num_classes)
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()

        # Define the GCNConv layer
        self.conv1 = GCNConv(3, 32)
        self.conv2 = GCNConv(32, 64)

    def forward(self, data):
        # Apply the GCNConv layers
        x = self.conv1(x, data.edge_index)
        x = F.relu(x)
        x = self.conv2(x,data.edge_index)
        x = F.relu(x)

        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = input_graphs[0]
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()